<a href="https://colab.research.google.com/github/pranavrajaram/datascience/blob/main/MLB_rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting the MLB world Series
The MLB is currently ongoing. The simplest way to predict a winner in the Fall, could be to check who is on top of the current standings. This, however, wouldn't take into account the strength of each team's schedule. We'll use the methods discussed in class instead. 

For this project, you are given a dataset containing all the games played this season through June. 

What you have to do:

*   Practice manipulating the dataset using Pandas. Check the introductory notebook `intro_to_pandas,ipynb`
*   Create the Massey matrix 
*   Solve the corresponding system to rank the teams using Massey's method 
*   Re-do the steps for the Colley method
*   Discuss any differences in the results, and explain why that happened. 
*   Pick a winner!


In [59]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/RodericGuigoCorominas/datascience/main/datasets/mlb_june_2022.csv'
df = pd.read_csv(url)


In [ ]:
# the code snippet below may be useful
all_teams = np.unique(df[['home','visitor']].to_numpy().flatten())
num_teams = len(all_teams)

print("There are",num_teams,"teams")
print(all_teams)

There are 30 teams
['Angels' 'Astros' 'Athletics' 'Blue Jays' 'Braves' 'Brewers' 'Cardinals'
 'Cubs' 'Diamondbacks' 'Dodgers' 'Giants' 'Guardians' 'Mariners' 'Marlins'
 'Mets' 'Nationals' 'Orioles' 'Padres' 'Phillies' 'Pirates' 'Rangers'
 'Rays' 'Red Sox' 'Reds' 'Rockies' 'Royals' 'Tigers' 'Twins' 'White Sox'
 'Yankees']


In [ ]:
# tip: it may be useful to use a dictionary to translate between teams and indices. This way, you can use the name of a team to represent the corresponding row or column
teams_dict = dict(zip(all_teams, range(num_teams)))

# 'Rangers' now corresponds to the index 20
teams_dict['Rangers']

20

In [60]:
# your code...

home_df = df.groupby(['home']).sum().reset_index()

home_df.columns = ['team','home_pf','home_pa']

away_df = df.groupby(['visitor']).sum().reset_index()

away_df.columns = ['team','away_pa','away_pf']

combined_df = pd.merge(home_df, away_df, on = ["team","team"])

combined_df['pf'] = combined_df['home_pf'] + combined_df['away_pf']
combined_df['pa'] = combined_df['home_pa'] + combined_df['away_pa']
final = combined_df[['team','pf','pa']]
final['diff'] = final['pf'] - final['pa']

#print(final)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
